# Análise de Churn

### Definição do problema

A retenção de clientes é um dos desafios mais críticos para empresas que operam em mercados competitivos. O churn, ou taxa de cancelamento de clientes, representa a proporção de clientes que deixam de utilizar um serviço dentro de um determinado período. Compreender os fatores que influenciam o churn é essencial para que as empresas desenvolvam estratégias eficazes de retenção, reduzindo perdas e maximizando a receita.

Neste projeto, realizaremos uma análise detalhada do comportamento dos clientes com base em um conjunto de dados contendo informações como idade, tempo de contrato, frequência de uso, total gasto, interações com o call center e outras variáveis relevantes. O objetivo é identificar padrões que diferenciam clientes que permanecem dos que cancelam, permitindo a criação de estratégias para minimizar o churn.

Por meio de estatísticas descritivas, visualizações de dados e modelagem preditiva, buscamos responder às seguintes perguntas-chave:
- Quais características dos clientes estão mais associadas ao churn?
- Clientes com maior tempo de contrato têm menor probabilidade de cancelar?
- Qual o impacto das interações com o call center na retenção?
- O valor total gasto influencia a decisão de cancelar o serviço?
- Ao final da análise, apresentaremos insights e recomendações práticas para reduzir o churn e melhorar a experiência do cliente.

## Dicionário de dados

**Dicionário de dados** para a sua tabela:  

| Nome da Variável            | Tipo       | Descrição |
|-----------------------------|-----------|-----------|
| **CustomerID**              | Inteiro    | Identificação única do cliente. |
| **idade**                   | Inteiro    | Idade do cliente (em anos). |
| **sexo**                    | Categórico | Gênero do cliente (ex.: "M" para masculino, "F" para feminino). |
| **tempo_como_cliente**       | Inteiro    | Tempo total (em meses) que o cliente está com a empresa. |
| **frequencia_uso**           | Numérico   | Frequência média de uso do serviço pelo cliente. |
| **ligacoes_callcenter**      | Inteiro    | Número de ligações realizadas para o call center. |
| **dias_atraso**             | Inteiro    | Número de dias que o cliente já atrasou pagamentos. |
| **assinatura**               | Categórico | Tipo de plano/assinatura do cliente (ex.: "Pré-pago", "Pós-pago", "Controle"). |
| **duracao_contrato**         | Inteiro    | Duração total do contrato do cliente (em meses). |
| **total_gasto**              | Numérico   | Valor total gasto pelo cliente durante o período com a empresa. |
| **meses_ultima_interacao**   | Inteiro    | Tempo (em meses) desde a última interação do cliente com a empresa. |
| **cancelou**                 | Binário    | Indica se o cliente cancelou o serviço (1 = Sim, 0 = Não). |


### 2. Exploração de dadoss

- Analise a distribuição das variáveis:
    - Idade dos clientes
    - Frequência de uso
    - Total gasto
    - Tempo como cliente
- Verifique possíveis outliers que possam distorcer a análise.

In [1]:
# importando a base dados
import pandas as pd
import plotly.express as px


In [2]:
# carregando os dados
tabela = pd.read_csv('dados/cancelamentos.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'dados/cancelamentos.csv'

In [ ]:
tabela.head()

In [ ]:
# verificando quantos registros e variáveis
tabela.shape

In [ ]:
# nome das colunas
tabela.columns

In [ ]:
# informações
tabela.info()

In [ ]:
# verificando se existem dados nulos
tabela.isnull().sum()

In [ ]:
# removendo os valores vazios
tabela = tabela.dropna()

In [ ]:
# conferindo a  nova quantidade de registros
tabela.shape

In [ ]:
# conferindo se os valores nulos foram removidos
tabela.isnull().sum()

In [ ]:
# removendo a coluna CustomerID
tabela.drop(columns='CustomerID', axis=1, inplace=True)

In [ ]:
# calculando a quantidade de cancelamentos 
print(tabela['cancelou'].value_counts())
print(tabela['cancelou'].value_counts(normalize=True).map("{:.2%}".format))


In [ ]:
# # Vamos através de graficos verificar a relação entre as variáveis e o cancelamento
# for variavel in tabela.columns:
#     print(f'{variavel.upper()}:')
#     #display(tabela.groupby(variavel).mean(numeric_only=True))
#     grafico = px.histogram(tabela, x=variavel, color='cancelou', color_discrete_map={1.0: "red", 0.0:"blue" })
#     grafico.show()
#     print(100*'--------')

## 3. Análise Univariada
- Estatísticas descritivas para entender cada variável.
- Histogramas e boxplots para variáveis numéricas.
- Contagem de categorias para variáveis categóricas (sexo, assinatura, cancelou).

#### Cenário 1: Excluir todos os planos Mensais

In [ ]:
## excluindo os clientes do plano mensal para verificar como ficaria a distribuição dos cancelamentos
tabela_filtro_assinatura = tabela.loc[tabela['duracao_contrato'] !='Monthly',:]
tabela_filtro_assinatura['cancelou'].value_counts()
tabela_filtro_assinatura['cancelou'].value_counts(normalize=True)

#### Cenário 2: Excluir todos os clientes que ligaram mais de 5 vezes

In [ ]:
# excluindo os clientes que ligaram mais de 5 vezes
tabela_filtro_ligacoes = tabela.loc[tabela['ligacoes_callcenter'] <= 5,:]
tabela_filtro_ligacoes['cancelou'].value_counts()
tabela_filtro_ligacoes['cancelou'].value_counts(normalize=True)

#### Cenário 3 : Excluir todos os clientes que atrasaram o pagamento mais de 20 dias

In [ ]:
# excluindo os clientes que atrasaram mais de 20 dias
tabela_filtro_dias_atraso = tabela.loc[tabela['dias_atraso'] <= 20,:]
tabela_filtro_dias_atraso['cancelou'].value_counts()
tabela_filtro_dias_atraso['cancelou'].value_counts(normalize=True)

#### Cenário 4: Excluir todos os clientes acima de 50 anos

In [ ]:
# excluindo os clientes acima de 50 anos
tabela_filtro_idade = tabela.loc[tabela['idade'] <= 50,:]
tabela_filtro_idade['cancelou'].value_counts()
tabela_filtro_idade['cancelou'].value_counts(normalize=True)

## 4. Análise Bivariada (Churn vs Outras Variáveis)
- Compare as características dos clientes que cancelaram (cancelou = 1) com os que não cancelaram (cancelou = 0).
- Idade média dos que cancelam vs. os que permanecem.
- Relação entre total gasto e cancelamento.
- Clientes que mais ligam para o call center têm maior taxa de churn?
- Clientes com mais tempo de contrato têm menor chance de cancelar?

#### Cenário 6: Excluir todos os clientes do plano mensal e que ligaram mais de 5 vezes 

In [ ]:
## excluindo os clientes do plano mensal para verificar como ficaria a distribuição dos cancelamentos
tabela_filtro_assinatura_e_ligacaoes = tabela.loc[(tabela['duracao_contrato'] !='Monthly') & (tabela['ligacoes_callcenter'] <= 5),:]
tabela_filtro_assinatura_e_ligacaoes['cancelou'].value_counts()
tabela_filtro_assinatura_e_ligacaoes['cancelou'].value_counts(normalize=True)

#### Cenário 7: Excluir todos os clientes do plano mensal e que atrasaram o pagamento mais de 20 dias

In [ ]:
#### excluindo os clientes do plano mensal e que atrasaram o pagamento mais de 20 dias
tabela_filtro_assinatura_e_dias_atraso = tabela.loc[(tabela['duracao_contrato'] !='Monthly') & (tabela['dias_atraso'] <= 20),:]
tabela_filtro_assinatura_e_ligacaoes['cancelou'].value_counts()
tabela_filtro_assinatura_e_dias_atraso['cancelou'].value_counts(normalize=True)

### Cenário 8: Excluir todos os clientes do plano mensal e com idade acima de 50 anos

In [ ]:
## excluindo os clientes do plano mensal e com idade acima de 50 anos
tabela_filtro_assinatura_e_idade = tabela.loc[(tabela['duracao_contrato'] !='Monthly') & (tabela['idade'] <= 50),:]
tabela_filtro_assinatura_e_idade['cancelou'].value_counts()
tabela_filtro_assinatura_e_idade['cancelou'].value_counts(normalize=True)

## 4. Análise Multivariáveis (Churn vs Outras Variáveis)
- Compare as características dos clientes que cancelaram (cancelou = 1) com os que não cancelaram (cancelou = 0).
- Idade média dos que cancelam vs. os que permanecem.
- Relação entre total gasto e cancelamento.
- Clientes que mais ligam para o call center têm maior taxa de churn?
- Clientes com mais tempo de contrato têm menor chance de cancelar?

#### Cenário 9: Excluir os clientes do plano mensal e que atrasaram o pagamento mais de 20 dias e que ligaram mais de 5 vezes 

In [ ]:
## excluindo os clientes do plano mensal e que atrasaram o pagamento mais de 20 dias e que ligaram mais de 5 vezes 
tabela_filtro_assinatura_e_dias_atraso_ligacoes = tabela.loc[(tabela['duracao_contrato'] !='Monthly') & (tabela['dias_atraso'] <= 20) & (tabela['ligacoes_callcenter'] <= 5) ,:]
tabela_filtro_assinatura_e_dias_atraso_ligacoes['cancelou'].value_counts()
tabela_filtro_assinatura_e_dias_atraso_ligacoes['cancelou'].value_counts(normalize=True)

In [ ]:
# CONCLUSÕES
# Se eliminássemos o plano Mensal, e reduzíssimos o tempo de atraso e a quantidade de ligações, teríamos uma expressíva queda do Churn 

## 5. Insights e Recomendações
- Identificamos que os clientes cancelaram seus planos de acordo com as situações abaixo:
    - Todos os clientes do plano Mensal cancelaram
    - Clientes com a partir de 51 anos, cancelaram
    - Clientes que realizaram, a partir de 5 ligações para o Call Center, cancelaram
    - Clientes que tiveram a partir de 21 dias de atraso, cancelam
    

- Taxa de cancelamento = 56,7 %
- Taxa de cancelamento excluindo os clientes do plano mensal = 46 %
- Taxa de cancelamento excluindo todos os clientes que ligaram mais de 5 vezes = 40,8 %
- Taxa de cancelamento excluindo os clientes que atrasaram o pagamento mais de 20 dias ou com idade acima de 50 anos = 46 %
- Taxa de cancelamento excluindo todos os clientes do plano mensal e que ligaram mais de 5 vezes = 30,5 %
- Taxa de cancelamento excluindo os clientes do plano mensal, os que atrasaram o pagamento mais de 20 dias e que ligaram mais de 5 vezes = 21,8 %

Sugestões:
- Excluir os planos mensais ou criar condições melhores para esses planos e se reduzíssimos o tempo de atraso e a quantidade de ligações, teríamos uma queda do Churn 
